# Basic Experiments

In this reference notebook we show how to define basic qubit tuneup experiments

# 0. General Imports and Definitions

## 0.1 Python Imports

In [ ]:
%config IPCompleter.greedy=True

# convenience import for all QCCS software functionality
from laboneq.simple import *

# helper import
from example_notebook_helper import *

# 1. Define Device Setup and Calibration

## 1.1 Define a Device Setup

Descriptor contains all information on instruments used, internal connections between instruments as well as wiring to the experiment

In [ ]:
descriptor="""\
instrument_list:
  HDAWG:
  - address: DEV8001
    uid: device_hdawg 
  UHFQA:
  - address: DEV2001    
    uid: device_uhfqa
  PQSC:  
  - address: DEV10001
    uid: device_pqsc
connections:
  device_hdawg:    
    - iq_signal: q0/drive_line
      ports: [SIGOUTS/0, SIGOUTS/1]        
    - iq_signal: q1/drive_line
      ports: [SIGOUTS/2, SIGOUTS/3]
    - rf_signal: q0/flux_line
      ports: [SIGOUTS/4]        
    - rf_signal: q1/flux_line
      ports: [SIGOUTS/5]              
    - to: device_uhfqa
      port: DIOS/0
  device_uhfqa:    
    - iq_signal: q0/measure_line
      ports: [SIGOUTS/0, SIGOUTS/1]        
    - acquire_signal: q0/acquire_line
    - iq_signal: q1/measure_line
      ports: [SIGOUTS/0, SIGOUTS/1]        
    - acquire_signal: q1/acquire_line
  device_pqsc:
    - to: device_hdawg
      port: ZSYNCS/0
"""

## 1.2 Define Calibration Settings

Modify the calibration on the device setup with known parameters for qubit control and readout - qubit control and readout frequencies, mixer calibration corrections

In [ ]:
# functions that modifies the calibration on a given device setup
def calibrate_devices(device_setup):
    ## qubit 0
    # calibration setting for drive line for qubit 0
    device_setup.logical_signal_groups["q0"].logical_signals["drive_line"].calibration = SignalCalibration(
        # oscillator settings - frequency and type of oscillator used to modulate the pulses applied through this signal line
        oscillator=Oscillator(
            uid="drive_q0_osc",
            frequency=1e8,
            modulation_type=ModulationType.HARDWARE),
        # mixer calibration settings to compensate for non-ideal mixer configuration
        mixer_calibration=MixerCalibration(
            voltage_offsets=[0.0, 0.0],
            correction_matrix=[
                [1.0, 0.0],
                [0.0, 1.0],
            ]
        ),
        # global and static delay of logical signal line: use to align pulses and compensate skew
        port_delay=0,       # applied to corresponding instrument node, bound to hardware limits
        delay_signal=0,     # inserted in sequencer code, bound to waveform granularity
    )
    # calibration setting for flux line for qubit 0
    device_setup.logical_signal_groups["q0"].logical_signals["flux_line"].calibration = SignalCalibration(
        # global and static delay of logical signal line: use to align pulses and compensate skew
        port_delay=0,       # applied to corresponding instrument node, bound to hardware limits
        delay_signal=0,     # inserted in sequencer code, bound to waveform granularity
    )
    # calibration setting for readout pulse line for qubit 0
    device_setup.logical_signal_groups["q0"].logical_signals["measure_line"].calibration = SignalCalibration(
        oscillator = Oscillator(
            uid="measure_q0_osc",
            frequency=1e8,
            modulation_type=ModulationType.SOFTWARE),
        delay_signal=0,     # inserted in sequencer code, bound to waveform granularity
        )
    # calibration setting for data acquisition line for qubit 0
    device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="acquire_osc",
            frequency=1e8,
            modulation_type=ModulationType.SOFTWARE),
        # delays the start of integration in relation to the start of the readout pulse to compensate for signal propagation time
        port_delay=10e-9,   # applied to corresponding instrument node, bound to hardware limits
        delay_signal=0,     # inserted in sequencer code, bound to waveform granularity
    )
    ## qubit 1
    # calibration setting for drive line for qubit 1
    device_setup.logical_signal_groups["q1"].logical_signals["drive_line"].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="drive_q1_osc",
            frequency=.5e8,
            modulation_type=ModulationType.HARDWARE),
        mixer_calibration=MixerCalibration(
            voltage_offsets=[0.0, 0.0],
            correction_matrix=[
                [1.0, 0.0],
                [0.0, 1.0],
            ]
        ),
        # global and static delay of logical signal line: use to align pulses and compensate skew
        port_delay=0,
        delay_signal=0,
    )
    # calibration setting for flux line for qubit 1
    device_setup.logical_signal_groups["q1"].logical_signals["flux_line"].calibration = SignalCalibration(
        # global and static delay of logical signal line: use to align pulses and compensate skew
        port_delay=0,
        delay_signal=0,
    )
    # calibration setting for readout pulse line for qubit 0
    device_setup.logical_signal_groups["q1"].logical_signals["measure_line"].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="measure_q1_osc",
            frequency=.5e8,
            modulation_type=ModulationType.SOFTWARE),
        delay_signal=0,
    )
    # calibration setting for data acquisition line for qubit 0
    device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="acquire_q1_osc",
            frequency=.5e8,
            modulation_type=ModulationType.SOFTWARE),
        # delays the start of integration in relation to the start of the readout pulse to compensate for signal propagation time
        port_delay=10e-9,
        delay_signal=0,
    )

## 1.3 Create Device Setup and Apply Calibration Settings

In [ ]:
# Function returning a calibrated device setup 
def create_device_setup():
    device_setup = DeviceSetup.from_descriptor(
        descriptor,
        server_host="my_ip_address",    # ip address of the LabOne dataserver used to communicate with the instruments
        server_port="8004",             # port number of the dataserver - default is 8004
        setup_name="my_QCCS_setup",     # setup name
    ) 
    calibrate_devices(device_setup)
    return device_setup

# create device setup
device_setup = create_device_setup()

In [ ]:
# use emulation mode - change, if running on hardware
use_emulation=True

# 2. Resonator Spectroscopy

Find the resonance frequency of the qubit readout resonator by looking at the transmission or reflection of a probe signal applied through the readout line

## 2.1 Define the Experiment

In [ ]:
# define sweep parameter - sweep over frequency of readout pulse
start = -100e6
stop = 100e6
count = 20

sweep_parameter = LinearSweepParameter(uid="sp1", start=start, stop=stop, count=count)

# define number of averages
average_exponent = 4  # used for 2^n averages, n=average_exponent, maximum: n = 17

# Create Experiment - uses only a readout pulse and a data acquisition line
exp = Experiment(
    uid="Resonator Spectroscopy",
    signals=[
        ExperimentSignal("measure"), 
        ExperimentSignal("acquire"),
        ],
)
## experimental pulse sequence
# outer loop - near-time sweep over readout pulse frequency
with exp.sweep(uid="sweep", parameter=sweep_parameter):
    # inner loop - real-time, sequential averaging in spectroscopy mode
    with exp.acquire_loop_rt(
            uid="shots",
            count=pow(2, average_exponent),
            averaging_mode=AveragingMode.SEQUENTIAL,
            acquisition_type=AcquisitionType.SPECTROSCOPY,
        ):
        # readout pulse and data acquisition
        with exp.section(uid="spectroscopy"):
            exp.play(
                signal="measure",
                pulse=pulse_library.const(length=1e-6, amplitude=1.0),
            )
            exp.acquire(
                signal="acquire",
                handle="ac_0",
                length=1e-6,
            )
        # relax time after readout - for signal processing and qubit relaxation to groundstate
        with exp.section(uid="relax"):
            exp.delay(signal="measure", time=1e-6)

In [ ]:
# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0['measure'] = SignalCalibration(
    oscillator = Oscillator(
        frequency=sweep_parameter,
        modulation_type=ModulationType.HARDWARE,
    )
)
# signal map for qubit 0
map_q0 = {
    "measure": "/logical_signal_groups/q0/measure_line",
    "acquire": "/logical_signal_groups/q0/acquire_line"
    }

## 2.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

# run experiment
my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot the simulated waveforms
plot_output_signals(my_results, uhfqa=0)

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Resonator Spectroscopy", compiled_exp)

# 3. Pulsed Qubit Spectroscopy

Find the resonance frequency of the qubit by looking at the change in resonator transmission when sweeping the frequency of a qubit excitation pulse

## 3.1 Define the Experiment

In [ ]:
## define pulses

# qubit drive pulse
const_iq_100ns = pulse_library.const(
    uid="const_iq_100ns", length=100e-9, amplitude=1.0
)
# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=400e-9, amplitude=1.0
)
# readout weights for integration
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)

## define calibration settings for readout - set on DeviceSetup as a  baseline calibration

lsg = device_setup.logical_signal_groups["q0"].logical_signals
# Use the result from resonator spectroscopy experiment to define readout frequency
lsg["measure_line"].calibration.oscillator.frequency = 100e6
# use software modulation to enable multiplexed qubit readout
lsg["measure_line"].oscillator.modulation_type = ModulationType.SOFTWARE
lsg["acquire_line"].calibration.oscillator.frequency = 100e6
lsg["acquire_line"].oscillator.modulation_type = ModulationType.SOFTWARE

lsg = device_setup.logical_signal_groups["q1"].logical_signals
# Use the result from resonator spectroscopy experiment to define readout frequency
lsg["measure_line"].calibration.oscillator.frequency = 50e6
lsg["measure_line"].oscillator.modulation_type = ModulationType.SOFTWARE
lsg["acquire_line"].calibration.oscillator.frequency = 50e6
lsg["acquire_line"].oscillator.modulation_type = ModulationType.SOFTWARE


In [ ]:
# define sweep parameter - sweep over the frequency of a qubit excitation pulse
start = 40e6
stop = 200e6
count = 40

sweep_parameter = LinearSweepParameter(uid="freq-qubit", start=start, stop=stop, count=count)

# define number of averages
average_exponent = 4  # used for 2^n averages, n=average_exponent, maximum: n = 17

# Create Experiment - no explicit mapping to qubit lines
exp = Experiment(
    uid="Qubit Spectroscopy",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental pulse sequence
# outer loop - near-time sweep over qubit drive frequency
with exp.sweep(uid="sweep", parameter=sweep_parameter, execution_type=ExecutionType.NEAR_TIME):
    # inner loop - real-time, sequential averaging in standard integration mode
    with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
        averaging_mode=AveragingMode.SEQUENTIAL, acquisition_type=AcquisitionType.INTEGRATION
        ):
        # qubit excitation pulse - frequency will be swept
        with exp.section(uid="qubit_excitation"):
            exp.play(signal="drive", pulse=const_iq_100ns)
        # readout and data acquisition
        with exp.section(uid="qubit_readout"):
            exp.reserve(signal="drive")
            # play readout pulse
            exp.play(signal="measure", pulse=readout_pulse)
            # signal data acquisition
            exp.acquire(
                signal="acquire",
                handle="ac_0",
                kernel=readout_weighting_function,
            )
        # relax time after readout - for signal processing and qubit relaxation to groundstate
        with exp.section(uid="relax"):
            exp.delay(signal="measure", time=1e-6)

In [ ]:
# define experiment calibration - sweep over qubit drive frequency
exp_calib = Calibration()
exp_calib["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=sweep_parameter,
        modulation_type=ModulationType.HARDWARE,
    )
)

# define signal maps for qubit 0
map_q0 = {
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}
# ... and qubit 1
map_q1 = {
    "drive": device_setup.logical_signal_groups["q1"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q1"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"],
}

## 3.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set calibration and signal map for qubit 0
exp.set_calibration(exp_calib)
exp.set_signal_map(map_q0)

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0 
my_results = session.run(exp)

In [ ]:
# set calibration and signal map for qubit 1
exp.set_calibration(exp_calib)
exp.set_signal_map(map_q1)

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 1
my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot the simulated waveforms
plot_output_signals(my_results, uhfqa=1, hdawg_drive=0)

In [ ]:
# plot measurement results
plot_result_2d(my_results, 'ac_0')

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Qubit Spectroscopy", compiled_exp)

# 4. Amplitude Rabi Experiment

Sweep the pulse amplitude of a qubit drive pulse to determine the ideal amplitudes for specific qubit rotation angles

## 4.1 Define the Experiment

In [ ]:
## define pulses

# qubit drive pulse - unit amplitude, but will be scaled with sweep parameter
x90 = pulse_library.gaussian(uid="x90", length=100e-9, amplitude=1.0)
# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=400e-9, amplitude=1.0
)
# readout integration weights
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=400e-9, amplitude=1.0
)

## define calibration settings for readout and drive - set here into the baseline calibration on DeviceSetup 

lsg = device_setup.logical_signal_groups["q0"].logical_signals
# Use the result from pulsed qubit spectroscopy experiment to define qubit drive frequency
lsg["drive_line"].calibration.oscillator.frequency = 100e6
lsg["drive_line"].oscillator.modulation_type = ModulationType.HARDWARE
# Use the result from resonator spectroscopy experiment to define readout frequency
lsg["measure_line"].calibration.oscillator.frequency = 100e6
# use software mode to enable multiplexed qubit readout 
lsg["measure_line"].oscillator.modulation_type = ModulationType.SOFTWARE
lsg["acquire_line"].calibration.oscillator.frequency = 100e6
lsg["acquire_line"].oscillator.modulation_type = ModulationType.SOFTWARE

lsg = device_setup.logical_signal_groups["q1"].logical_signals
# Use the result from pulsed qubit spectroscopy experiment to define qubit drive frequency
lsg["drive_line"].calibration.oscillator.frequency = 50e6
lsg["drive_line"].oscillator.modulation_type = ModulationType.HARDWARE
# Use the result from resonator spectroscopy experiment to define readout frequency
lsg["measure_line"].calibration.oscillator.frequency = 50e6
lsg["measure_line"].oscillator.modulation_type = ModulationType.SOFTWARE
lsg["acquire_line"].calibration.oscillator.frequency = 50e6
lsg["acquire_line"].oscillator.modulation_type = ModulationType.SOFTWARE

In [ ]:
# set up sweep parameter - drive amplitude
start = 0.1
stop = 1
count = 10
sweep_parameter = LinearSweepParameter(uid="amplitude", start=start, stop=stop, count=count)

# number of averages
average_exponent = 1  # used for 2^n averages, n=average_exponent, maximum: n = 17

# Create Experiment
exp = Experiment(
    uid="Amplitude Rabi",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental pulse sequence
# outer loop - real-time, cyclic averaging in standard integration mode
with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC, acquisition_type=AcquisitionType.INTEGRATION
    ):
    # inner loop - real-time sweep of qubit drive pulse amplitude
    with exp.sweep(uid="sweep", parameter=sweep_parameter):
            # qubit excitation - pulse amplitude will be swept
            with exp.section(uid="qubit_excitation"):
                exp.play(signal="drive", pulse=x90, amplitude=sweep_parameter)
            # qubit readout pulse and data acquisition
            with exp.section(uid="qubit_readout"):
                exp.reserve(signal="drive")
                # play readout pulse
                exp.play(signal="measure", pulse=readout_pulse)
                # signal data acquisition
                exp.acquire(
                    signal="acquire",
                    handle="ac_0",
                    kernel=readout_weighting_function,
                )
            # relax time after readout - for signal processing and qubit relaxation to groundstate
            with exp.section(uid="relax"):
                exp.delay(signal="measure", time=1e-6)

In [ ]:
# define signal maps for qubit 0
map_q0 = {
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}
# ... - and qubit 1
map_q1 = {
    "drive": device_setup.logical_signal_groups["q1"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q1"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"],
}

## 4.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set signal map to qubit 0
exp.set_signal_map(map_q0)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0
my_results = session.run(exp, do_simulation=True)

In [ ]:
# set signal map to qubit 1
exp.set_signal_map(map_q1)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 1
my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot simulated output signals
plot_output_signals(my_results, uhfqa=1, hdawg_drive=0)

In [ ]:
# plot measurement results
plot_result_2d(my_results, 'ac_0')

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Amplitude Rabi", compiled_exp)

# 5. Ramsey Experiment

Sweep the delay between two slightly detuned pi/2 pulses to determine the qubit dephasing time as well as fine calibration its excited state frequency

## 5.1 Define the Experiment

In [ ]:
## define pulses

# qubit drive pulse - use amplitude calibrated by amplitude Rabi experiment
x90 = pulse_library.gaussian(uid="x90", length=100e-9, amplitude=0.66)
# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=400e-9, amplitude=1.0
)
# readout integration weights
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=400e-9, amplitude=1.0
)

## calibration settings for readout and drive already set in the baseline calibration on DeviceSetup

In [ ]:
# set up sweep parameter - delay between pi/2 pulses
start = 0.0
stop = 1000e-9
count = 10

sweep_parameter = LinearSweepParameter(uid="delay", start=start, stop=stop, count=count)

# number of averages
average_exponent = 1  # used for 2^n averages, n=average_exponent, maximum: n = 17

# Create Experiment
exp = Experiment(
    uid="Ramsey",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental pulse sequence
# outer loop - real-time, cyclic averaging in standard integration mode
with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC, acquisition_type=AcquisitionType.INTEGRATION
    ):
    # inner loop - real-time sweep over delay between qubit pulses
    with exp.sweep(uid="sweep", parameter=sweep_parameter):
        # qubit drive pulses - use right-aligned, constant length section to optimize overall experimental sequence
        # another way of implementation with a right-aligned sweep section and repetition time AUTO is shown in T1 experiment below
        with exp.section(uid="qubit_excitation", length=stop+2*x90.length, alignment=SectionAlignment.RIGHT):
            exp.play(signal="drive", pulse=x90)
            exp.delay(signal="drive", time=sweep_parameter)
            exp.play(signal="drive", pulse=x90)
        # qubit readout pulse and data acquisition
        with exp.section(uid="qubit_readout"):
            exp.reserve(signal="drive")
            # play readout pulse
            exp.play(signal="measure", pulse=readout_pulse)
            # signal data acquisition
            exp.acquire(
                signal="acquire",
                handle="ac_0",
                kernel=readout_weighting_function,
            )
        # relax time after readout - for signal processing and qubit relaxation to groundstate
        with exp.section(uid="relax"):
            exp.delay(signal="measure", time=1e-6)

In [ ]:
# define signal maps for qubit 0
map_q0 = {
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}
# ... and qubit 1
map_q1 = {
    "drive": device_setup.logical_signal_groups["q1"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q1"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"],
}

## 5.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# map exp to qubit 0
exp.set_signal_map(map_q0)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run on qubit 0
my_results = session.run(exp, do_simulation=True)

In [ ]:
# map exp to qubit 1
exp.set_signal_map(map_q1)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run on qubit 1
my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot the simulated waveforms
plot_output_signals(my_results, uhfqa=1, hdawg_drive=0)

In [ ]:
# plot measurement results
plot_result_2d(my_results, 'ac_0')

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey", compiled_exp)

# 6. T1 Experiment

Sweep the delay between a qubit excitation pulse and the readout to measure the energy relaxation time of the qubit

## 6.1 Define the Experiment

In [ ]:
## define pulses

# qubit drive pulse - use amplitude calibrated by amplitude Rabi experiment
x180 = pulse_library.gaussian(uid="x180", length=100e-9, amplitude=0.66)
# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=400e-9, amplitude=1.0
)
# readout integration weights
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=400e-9, amplitude=1.0
)

## calibration settings for readout and drive already set in the baseline calibration on DeviceSetup

In [ ]:
# sweep parameter - delay before readout / after pi pulse
count = 10
start = 0
stop = 1e-6
sweep_parameter = LinearSweepParameter(uid="delay", start=start, stop=stop, count=count)

# number of averages
average_exponent = 10  # used for 2^n averages, n=average_exponent, maximum: n = 17

# Create Experiment
exp = Experiment(
    uid="T1 experiment",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental pulse sequence
# outer loop - real-time, cyclic averaging in standard integration mode
with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC, acquisition_type=AcquisitionType.INTEGRATION,
    repetition_mode=RepetitionMode.AUTO # compiler sets repetition time to shortest possible constant time
    ):
    # inner loop - sweep over delay between qubit excitation and readout pulse
    # right alignment makes sure the readout section follows a fixed timing, being the last element in each shot
    with exp.sweep(uid="sweep", parameter=sweep_parameter, alignment=SectionAlignment.RIGHT):
        # qubit drive pulse followed by variable delay
        with exp.section(uid="qubit_excitation", alignment=SectionAlignment.RIGHT):
            exp.play(signal="drive", pulse=x180)
            exp.delay(signal="drive", time=sweep_parameter)
        # qubit readout pulse and data acquisition
        with exp.section(uid="qubit_readout"):
            exp.reserve(signal="drive")
            # play readout pulse
            exp.play(signal="measure", pulse=readout_pulse)
            # signal data acquisition
            exp.acquire(
                signal="acquire",
                handle="ac_0",
                kernel=readout_weighting_function,
            )
        # relax time after readout - for signal processing and qubit relaxation to groundstate
        with exp.section(uid="relax"):
            exp.delay(signal="measure", time=1e-6)

In [ ]:
# define signal maps for qubit 0
map_q0 = {
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}
# ... and qubit 1
map_q1 = {
    "drive": device_setup.logical_signal_groups["q1"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q1"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"],
}

## 6.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# map exp to qubit 0
exp.set_signal_map(map_q0)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run on qubit 0
my_results = session.run(exp, do_simulation=True)

In [ ]:
# map exp to qubit 1
exp.set_signal_map(map_q1)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run on qubit 1
my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot the simulated waveforms
plot_output_signals(my_results, uhfqa=1, hdawg_drive=0)

In [ ]:
# plot measurement results
plot_result_2d(my_results, 'ac_0')

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("T1", compiled_exp)

# 7. Ramsey with Sampled Pulse Definition

Sweep the delay between two slightly detuned pi/2 pulses to determine the qubit dephasing time as well as fine calibration its excited state frequency

Here use a user defined pulse, given as complex valued numpy array 

## 7.1 Pulse definition

In [ ]:
## define pulses

# qubit drive pulse as sampled complex pulse
x90_length = 100e-9
num_samples = round((x90_length * 2.4e9)) # 2.4 GS/s is the sampling rate of the HDAWG which plays the qubit drive pulse
samples_complex = np.transpose(np.array([np.arange(num_samples) *(1/num_samples) , np.arange(num_samples) *(1/num_samples)]))

x90 = pulse_library.sampled_pulse_complex(samples=samples_complex, uid="x90")
# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=400e-9, amplitude=1.0
)
# readout integration weights
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)

## calibration settings for readout and drive already set in the baseline calibration on DeviceSetup

## 7.2 Define the Experiment

In [ ]:
# set up sweep parameter - delay between pi/2 pulses
start = 0.0
stop = 1000e-9
count = 10

sweep_parameter = LinearSweepParameter(
    uid="delay", start=start, stop=stop, count=count)

# number of averages
average_exponent = 1  # used for 2^n averages, n=average_exponent, maximum: n = 17

# Create Experiment
exp = Experiment(
    uid="Ramsey",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental sequence
# outer loop - real-time, cyclic averaging in standard integration mode
with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC, acquisition_type=AcquisitionType.INTEGRATION
    ):
    # inner loop - sweep over delay between qubit drive pulses
    with exp.sweep(uid="sweep", parameter=sweep_parameter):
        # qubit excitation pulses - use right-aligned, constant length section to optimise pulse timings
        with exp.section(uid="qubit_excitation", length=stop + 2 * x90_length, alignment=SectionAlignment.RIGHT):
            exp.play(signal="drive", pulse=x90)
            exp.delay(signal="drive", time=sweep_parameter)
            exp.play(signal="drive", pulse=x90)
        # qubit readout pulse and data acquisition 
        with exp.section(uid="qubit_readout"):
            exp.reserve(signal="drive")
            # play readout pulse
            exp.play(signal="measure", pulse=readout_pulse)
            # signal data acquisition
            exp.acquire(
                signal="acquire",
                handle="ac_0",
                kernel=readout_weighting_function,
            )
        # relax time after readout - for signal processing and qubit relaxation to groundstate
        with exp.section(uid="relax"):
            exp.delay(signal="measure", time=1e-6)

In [ ]:
# define signal maps for different qubits
map_q0 = {
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}

map_q1 = {
    "drive": device_setup.logical_signal_groups["q1"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q1"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"],
}

##  7.3 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set experiment to map to qubit 0
exp.set_signal_map(map_q0)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0
my_results = session.run(exp, do_simulation=True)

In [ ]:
# set experiment to map to qubit 1
exp.set_signal_map(map_q1)

# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 1
my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot the simulated waveforms
plot_output_signals(my_results, uhfqa=1, hdawg_drive=0)

In [ ]:
# plot measurement results
plot_result_2d(my_results, 'ac_0')

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("T1", compiled_exp)

# 8. Pulsed Qubit Spectroscopy for flux dependent qubit

Find the flux dependent resonance frequency of a qubit by looking at the change in resonator transmission when sweeping the frequency of a qubit excitation pulse

## 8.1 Define the Experiment

In [ ]:
## define pulses

# flux pulse - applied during whole experimental pulse sequence
const_flux = pulse_library.const(
    uid="const_flux", length=600e-9, amplitude=1.0
)
# qubit drive pulse
const_iq_100ns = pulse_library.const(
    uid="const_iq_100ns", length=100e-9, amplitude=1.0
)
# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=400e-9, amplitude=1.0
)
# readout weights for integration
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)

## define calibration settings for readout - set on DeviceSetup as a  baseline calibration

lsg = device_setup.logical_signal_groups["q0"].logical_signals
# Use the result from resonator spectroscopy experiment to define readout frequency
lsg["measure_line"].calibration.oscillator.frequency = 100e6
lsg["measure_line"].oscillator.modulation_type = ModulationType.SOFTWARE
lsg["acquire_line"].calibration.port_delay=150e-9   # allow readout signal to settle before data acquisition
lsg["acquire_line"].calibration.oscillator.frequency = 100e6
lsg["acquire_line"].oscillator.modulation_type = ModulationType.SOFTWARE

lsg = device_setup.logical_signal_groups["q1"].logical_signals
# Use the result from resonator spectroscopy experiment to define readout frequency
lsg["measure_line"].calibration.oscillator.frequency = 50e6
lsg["measure_line"].oscillator.modulation_type = ModulationType.SOFTWARE
lsg["acquire_line"].calibration.port_delay=150e-9   # allow readout signal to settle before data acquisition
lsg["acquire_line"].calibration.oscillator.frequency = 50e6
lsg["acquire_line"].oscillator.modulation_type = ModulationType.SOFTWARE


In [ ]:
# define sweep parameter - sweep over the frequency of a qubit excitation pulse
start = 40e6
stop = 200e6
count = 40

sweep_parameter = LinearSweepParameter(uid="freq-qubit", start=start, stop=stop, count=count)

# define flux sweep parameters
flux_count = 20

flux_sweep = LinearSweepParameter(uid="flux_qubit", start=0, stop=1, count=flux_count)

# define number of averages
average_exponent = 4  # used for 2^n averages, n=average_exponent, maximum: n = 17

In [ ]:
# Create Experiment - no explicit mapping to qubit lines
exp = Experiment(
    uid="Qubit Flux Spectroscopy",
    signals=[
        ExperimentSignal("flux"),
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)
## experimental pulse sequence
# outer loop - near-time sweep over qubit drive frequency
with exp.sweep(uid="sweep", parameter=sweep_parameter, execution_type=ExecutionType.NEAR_TIME):
    # inner loop - real-time, sequential averaging in standard integration mode
    with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
        averaging_mode=AveragingMode.CYCLIC, acquisition_type=AcquisitionType.INTEGRATION
        ):
        # inner loop - adjust flux bias to qubit
        with exp.sweep(uid="flux_sweep", parameter=flux_sweep):
            with exp.section(uid="flux bias"):
                exp.play(signal="flux", pulse=const_flux)
            # qubit excitation pulse - frequency will be swept
            with exp.section(uid="qubit_excitation"):
                # allow for transients to settle
                exp.delay(signal="drive", time=100e-9)
                # play excitation pulse
                exp.play(signal="drive", pulse=const_iq_100ns)
            # readout and data acquisition
            with exp.section(uid="qubit_readout"):
                exp.reserve(signal="drive")
                # play readout pulse
                exp.play(signal="measure", pulse=readout_pulse)
                # signal data acquisition
                exp.acquire(
                    signal="acquire",
                    handle="ac_0",
                    kernel=readout_weighting_function,
                )
            # relax time after readout - for signal processing and qubit relaxation to groundstate
            with exp.section(uid="relax"):
                exp.delay(signal="measure", time=1e-6)

In [ ]:
# define experiment calibration - sweep over qubit drive frequency
exp_calib = Calibration()
exp_calib["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=sweep_parameter,
        modulation_type=ModulationType.HARDWARE,
    )
)

# define signal maps for qubit 0
map_q0 = {
    "flux": device_setup.logical_signal_groups["q0"].logical_signals["flux_line"],
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}
# ... and qubit 1
map_q1 = {
    "flux": device_setup.logical_signal_groups["q1"].logical_signals["flux_line"],
    "drive": device_setup.logical_signal_groups["q1"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q1"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"],
}

## 8.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set calibration and signal map for qubit 0
exp.set_calibration(exp_calib)
exp.set_signal_map(map_q0)

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0 
my_results = session.run(exp, do_simulation=True)

In [ ]:
# set calibration and signal map for qubit 1
exp.set_calibration(exp_calib)
exp.set_signal_map(map_q1)

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 1
my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot the simulated waveforms
plot_output_signals(my_results, uhfqa=2, hdawg_drive=0, hdawg_flux=1)

In [ ]:
# plot measurement results
plot_result_3d(my_results, 'ac_0')

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Qubit Flux Spectroscopy", compiled_exp)

# 9. Flux-Scope Experiment

Experiment to characterise the distortions of flux pulses due to the imperfect signal lines, following chapter 4.4.3 in https://www.research-collection.ethz.ch/handle/20.500.11850/153681

## 9.1 Define the Experiment

In [ ]:
# Pulse definitions

# qubit excitation pulse - amplitude such that if pulse is resonant, results in pi rotation
x180 = pulse_library.gaussian(uid="x180", length=20e-9, amplitude=0.66)

# flux pulse - constant length and amplitude
flux_pulse = pulse_library.const(uid="flux_pulse", length=400e-9, amplitude=0.5)

# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=250e-9, amplitude=1.0
)
# readout weights for integration
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)

# assuming all calibration settings are already correct


In [ ]:
# define sweep parameters

# qubit excitation pulse frequency sweep
start_freq = 40e6
stop_freq = 200e6
count_freq = 11

sweep_frequency = LinearSweepParameter(uid="freq-qubit", start=start_freq, stop=stop_freq, count=count_freq)

# sweep delay between start of flux pulse and start of qubit excitation pulse
start_delay = 0
stop_delay = flux_pulse.length - x180.length
count_delay = 11

sweep_delay = LinearSweepParameter(uid="delay", start=start_delay,  stop=stop_delay, count=count_delay)

# define number of averages
average_exponent = 10  # used for 2^n averages, n=average_exponent, maximum: n = 19


In [ ]:
# Create Experiment
exp = Experiment(
    "Flux Scope",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("flux"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)

## experimental pulse sequence
# outer sweep - qubit excitation frequency
with exp.sweep(uid="frequency_sweep", parameter=sweep_frequency, execution_type=ExecutionType.NEAR_TIME):
    # real-time acquisiton loop in integration mode
    with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
        averaging_mode=AveragingMode.CYCLIC, acquisition_type=AcquisitionType.INTEGRATION
    ):
        # inner sweep - delay between start of qubit excitation pulse and start of flux pulse
        with exp.sweep(uid="sweep", parameter=sweep_delay):
            # flux pulse
            with exp.section(uid="qubit_detuning"): 
                exp.play(signal="flux", pulse=flux_pulse)
            # qubit excitation pulse - delayed with respect to start of flux pulse
            with exp.section(uid="qubit_excitation"):
                exp.delay(signal="drive", time=sweep_delay)
                exp.play(signal="drive", pulse=x180)
            # readout and data acquisition
            with exp.section(uid="qubit_readout"):
                exp.reserve(signal="drive")
                exp.reserve(signal="flux")
                exp.play(signal="measure", pulse=readout_pulse)
                exp.acquire(
                    signal="acquire",
                    handle="ac_0",
                    kernel=readout_weighting_function,
                )
            # relax time after readout - for signal processing and qubit relaxation to groundstate
            with exp.section(uid="relax"):
                exp.delay(signal="measure", time=1e-6)  

In [ ]:
# define experiment calibration - sweep over qubit drive frequency
exp_calib = Calibration()
exp_calib["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=sweep_frequency,
        modulation_type=ModulationType.HARDWARE,
    )
)

# define signal maps for qubit 0
map_q0 = {
    "flux": device_setup.logical_signal_groups["q0"].logical_signals["flux_line"],
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}
# ... and qubit 1
map_q1 = {
    "flux": device_setup.logical_signal_groups["q1"].logical_signals["flux_line"],
    "drive": device_setup.logical_signal_groups["q1"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q1"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"],
}

## 9.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set calibration and signal map for qubit 0
exp.set_calibration(exp_calib)
exp.set_signal_map(map_q0)

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)

# run experiment on qubit 0 
my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot the simulated waveforms
plot_output_signals(my_results, uhfqa=2, hdawg_drive=0, hdawg_flux=1)

In [ ]:
# plot measurement results
plot_result_3d(my_results, 'ac_0')

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Flux Scope Experiment", compiled_exp)

# 10. Cryoscope experiment

Alternate experiment to characterise the pulse distortions from line impedance, following https://arxiv.org/pdf/1907.04818.pdf - Ramsey sequence with fixed timing and variable flux pulse in between - sweeping flux pulse length and amplitude

Please note that number of sweep steps is currently (version 0.65) limited by the instruction limit on the quantum analyzer. This is due to incomplete compression of its sequencerC code.

## 10.1 Define the Experiment

In [ ]:
## define standard pulses

# qubit drive pulse - pi/2 rotation
x90 = pulse_library.gaussian(
    uid="x90", length=100e-9, amplitude=0.66
)
# readout drive pulse
readout_pulse = pulse_library.const(
    uid="readout_pulse", length=250e-9, amplitude=1.0
)
# readout weights for integration
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)

# assuming all calibration settings are already correct

In [ ]:
# define near time sweep parameter - pulse length
length_start = 10e-9
length_stop = 100e-9
length_count = 5    # known issue: limited to max. 30 steps

# flux_lengths = np.linspace(length_start, length_stop, length_count)
flux_lengths = LinearSweepParameter(start = length_start, stop = length_stop, count = length_count)

# define a dict of pulses based on the pulse length
flux_pulses = {}
for length in flux_lengths.values:
    flux_name = 'flux_' + str(round(1e9 * length))
    flux_pulses[flux_name] = pulse_library.const(uid=flux_name, length=length, amplitude=1.0)

In [ ]:
# define flux amplitude sweep parameter
flux_amp_count = 3

flux_amp_sweep = LinearSweepParameter(uid="flux_qubit", start=0.1, stop=1, count=flux_amp_count)

# define number of averages
average_exponent = 1  # used for 2^n averages, n=average_exponent, maximum: n = 17

In [ ]:
# Create Experiment - no explicit mapping to qubit lines
exp = Experiment(
    uid="Cryoscope experiment",
    signals=[
        ExperimentSignal("flux"),
        ExperimentSignal("drive"),
        ExperimentSignal("measure"),
        ExperimentSignal("acquire"),
    ],
)

with exp.acquire_loop_rt(uid="shots", count=pow(2, average_exponent),
    averaging_mode=AveragingMode.CYCLIC, acquisition_type=AcquisitionType.INTEGRATION
):
    with exp.sweep(parameter=flux_amp_sweep):
        for length in flux_lengths.values:
            flux_name = 'flux_' + str(round(1e9 * length))
            with exp.section(uid="shot_"+flux_name):                
                # qubit excitation pulses - Ramsey with fixed timing 
                with exp.section():
                    # play first Ramsey excitation pulse
                    exp.play(signal="drive", pulse=x90)
                    exp.delay(signal="drive", time=2*length_stop)
                    # play second Ramsey excitation pulse
                    exp.play(signal="drive", pulse=x90)
                # interleaved flux pulse with variable length and amplitude
                with exp.section():
                    # delay while first Ramsey pulse is played
                    exp.delay(signal="flux", time=x90.length)
                    # flux pulse
                    exp.play(signal="flux", pulse=flux_pulses[flux_name], amplitude=flux_amp_sweep)
                # readout and data acquisition
                with exp.section():
                    exp.reserve(signal="drive")
                    exp.reserve(signal="flux")
                    # play readout pulse
                    exp.play(signal="measure", pulse=readout_pulse)
                    # signal data acquisition
                    exp.acquire(
                        signal="acquire",
                        handle="ac_0",
                        kernel=readout_weighting_function,
                    )
                # relax time after readout - for signal processing and qubit relaxation to groundstate
                with exp.section():
                    exp.delay(signal="measure", time=1e-6)

In [ ]:
# define signal maps for qubit 0
map_q0 = {
    "flux": device_setup.logical_signal_groups["q0"].logical_signals["flux_line"],
    "drive": device_setup.logical_signal_groups["q0"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q0"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q0"].logical_signals["acquire_line"],
}
# ... and qubit 1
map_q1 = {
    "flux": device_setup.logical_signal_groups["q1"].logical_signals["flux_line"],
    "drive": device_setup.logical_signal_groups["q1"].logical_signals["drive_line"],
    "measure": device_setup.logical_signal_groups["q1"].logical_signals["measure_line"],
    "acquire": device_setup.logical_signal_groups["q1"].logical_signals["acquire_line"],
}

## 10.2 Run the Experiment and Plot the Measurement Results and Pulse Sequence

In [ ]:
# set calibration and signal map for qubit 0
exp.set_signal_map(map_q0)

# create a session and connect to it
session = Session(device_setup=device_setup)
session.connect(do_emulation=use_emulation)
session.max_simulation_time = 5e-6

my_results = session.run(exp, do_simulation=True)

In [ ]:
# Plot the simulated waveforms
plot_output_signals(my_results, uhfqa=2, hdawg_drive=0, hdawg_flux=1)

In [ ]:
# plot measurement results
plot_result_3d(my_results, 'ac_0')

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
# compiled_exp = session.compiled_experiment
show_pulse_sheet("Cryoscope", session.compiled_experiment)